In [2]:
import pandas as pd
import os

for dataset_name in ['eu', 'en', 'jp']:
    df = pd.read_excel(f'../step4-llm_select/results/{dataset_name}/{dataset_name}_results.xlsx').drop('task_top5', axis=1)
    task_candidate = pd.read_csv(f'../step4-llm_select/data/{dataset_name}/seed0/input0/task_candidate.csv', sep='\t')
    task_candidate['bert_task'] = task_candidate.apply(lambda x: [x[f"task{i}"] for i in range(100)], axis=1)
    task_candidate = task_candidate[['job_title', 'job_description', 'bert_task']]
    df = pd.merge(df, task_candidate, on=['job_title', 'job_description'])
    annotated_df = pd.read_csv(f'../step4-llm_select/results/{dataset_name}/{dataset_name}_gpt-llm-bert.csv')[['job_title', 'job_description', 'task']]
    tag_list = annotated_df['job_description'].tolist()
    annotated_df = pd.concat([annotated_df, df[annotated_df.columns]]).drop_duplicates('job_description')
    df = pd.merge(annotated_df, df.drop('task', axis=1), on=['job_title', 'job_description'])
    df['tag'] = df.apply(lambda x: 1 if x['job_description'] in tag_list else 0, axis=1)
    df = df[(df['tag'] == 1) | (df['score']>40) & (df['sim']>0.9)]
    df.to_csv(f'data/{dataset_name}.csv', index=None)
df

,job_title,job_description,task,sim,score,bert_task,tag
0,PdM(プロダクトマネジャー、グロースマーケター)/データアナリスト/インターネット/Web...,担当するKPIを改善するグロース施策の立案といリード,ビジネス戦略または市場戦略を策定します。,97.346118,49,"[コンプライアンス事項に関連するレポートを作成します。, 他のユーザーと協力してマーケティン...",1
1,プロダクトマーケティングマネージャー(PMM/プロダクト企画)※マネージャー候補(東京)【楽...,推進 ※開発の仕様策定、スコープ定義、プロジェクトマネジメントなどのプロダクトマネジメント...,ビジネス戦略または市場戦略を策定します。,96.699004,49,"[他のユーザーと協力してマーケティング戦略を開発または実行します。, ビジネスや運営上の問題...",1
2,ファンドマネージャー/その他金融・保険系,として、ヤマワケの案件における事業者様側との調整や組成後の各ファンドの進捗確認等、案件全体の...,ビジネス戦略または市場戦略を策定します。,96.251935,50,"[他のユーザーと協力してマーケティング戦略を開発または実行します。, ビジネスや運営上の問題...",1
3,プロダクトマネージャ候補/通信キャリア・データセンター・ISP,新規/既存プロダクトの開発を、サービス開発チームと連携して推進して頂きます,ビジネス戦略または市場戦略を策定します。,98.416990,50,"[顧客またはサービスプロバイダーと契約を交渉します。, プロジェクト情報を他の人に伝えます。...",1
4,【建設クラウド】[プロダクトマネージャー(シニア)]業界シェアNo.1のVerticalSa...,ズの各プロダクトと連動した新規プロダクト企画や、各プロダクトを連携した大規模プロジェクトをリ...,ビジネス戦略または市場戦略を策定します。,98.186588,50,"[ビジネスまたは財務データを分析します。, 顧客またはサービスプロバイダーと契約を交渉します...",1
...,...,...,...,...,...,...,...
789,デザイン業務(グラフィックデザイン、DTPデザイン、webデザイン)(石川),グラフィック、DTP、Ｗebのデザイン等の幅広いデザイン業務,ウェブサイトのコンテンツを更新します。,98.508679,41,"[統合コンピュータ システムを設計します。, パフォーマンスを向上させるためにソフトウェア ...",0
790,データ入力,官公庁でのお仕事です,法律や規制について他の人に知らせます。,98.317589,41,"[顧客またはサービスプロバイダーと契約を交渉します。, ビジネス戦略、実践、またはポリシーに...",0
791,一般事務・エクセルでのデータ入力,日の勤務です,顧客の苦情や問題を解決します。,97.938768,41,"[同僚と協議して機器の問題を解決します。, 判例やその他の法的情報から、事例への影響を特定し...",0
792,データサイエンティスト/インターネット/Webサービス・ASP,データからビジネス課題を明確にし、分析アプローチの策定、分析、レポーティング,ビジネスまたは財務データを分析します。,95.774825,41,"[ビジネスや運営上の問題について他の人にアドバイスします。, 市民プロジェクトや公共政策を評...",0
